In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from mlxtend.plotting import plot_decision_regions
from sklearn.linear_model import LogisticRegression


In [ ]:
tb_hotel_train = pd.read_csv("data/tb_hotel_train_clean.csv")
tb_hotel_test = pd.read_csv("data/tb_hotel_test_clean.csv")

tb_hotel_train.head()


In [ ]:
x_var = ["lead_time", "adr"]
X_train = tb_hotel_train[x_var]
X_test = tb_hotel_test[x_var]
y_train = tb_hotel_train["is_cancelled"]
y_test = tb_hotel_test["is_cancelled"]


# Regressão Logística

Antes de mergulharmos nos algoritmos `KNearestNeighborsClassifier` e `DecisionTreeClassifier`, vamos relembrar o que vimos na aula passada sobre o método mais simples de classificação: a **regressão logística**.

In [ ]:
log_fit = LogisticRegression()
log_fit.fit(X_train, y_train)


Um modelo de classificação prevê a **probabilidade** de um evento acontecer - nossa regressão está prevendo a probabilidade de uma reserva ser cancelada dado um `lead time` e `average daily rate`. Podemos visualizar essas probabilidades utilizando o método `.predict_proba()` (lembrando que ele retorna os **2 vetores de probabilidade**, a chance de *não-cancelamento* e *cancelamento*!)

In [ ]:
tb_log_fit = X_train.copy()
tb_log_fit["pred_prob"] = log_fit.predict_proba(X_train)[:, 1]
tb_log_fit["is_cancelled"] = y_train
sns.scatterplot(data=tb_log_fit, x="lead_time", y="pred_prob", hue="adr")


In [ ]:
fig = plt.figure(figsize=(10, 10))
plot_decision_regions(
    np.array(tb_log_fit[["lead_time", "adr"]]),
    np.array(tb_log_fit["is_cancelled"]),
    log_fit,
    scatter_kwargs={"alpha": 0.001},
)


Podemos utilizar o método `.predict()` para gerar previsões de classificação - uma previsão binária construída a partir do *thresholding* da probabilidade de cancelamento:

In [ ]:
y_test_log = log_fit.predict(X_test)
y_test_log[0:10]


In [ ]:
tb_pred_log = pd.DataFrame({"y_real": y_test, "y_previsto": y_test_log})
tb_pred_log.head()


## Medindo o Erro de Previsão

Na aula passada introduzimos o problema de como calcular a precisão de um modelo de classificação. Não poddemos **medir a acurácia da função de probabilidade**: *ela não é observada*. Precisamos utilizar as previsões de classificação para medir o erro do modelo. Para tanto, vamos ver algumas **métricas** diferentes para modelos de classificação.

### Acurácia

A **acurácia** é a medida mais simples de erro dos modelos de classficação:

$$Acc. = \frac{\text{Núm. de Previsões Corretas}}{\text{Núm. de Previsões Totais}}$$

In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
accuracy_score(y_test, y_test_log)


A acúracia é uma medida razoável quando nossa variável resposta não é *desbalanceada*, ou seja, contém o mesmo número de observações em cada classe.

Não é o caso em nosso problema:

In [ ]:
print(f"Núm. de Cancelamentos no Teste: {np.sum(y_test)}")
print(f"Núm. de Reservas no Teste: {len(y_test)}")


### Matriz de Confusão

A maior parte dos problemas de classficação **não são balanceados**: em geral a classe que desejamos prever é a classe minoritária (por exemplo, fraudes em um banco ou então churn em um serviço de assinatura). Nesses casos a acurácia **não é uma boa medida de erro**!

Além disso, muitas vezes, problemas de classificação tem **custos** diferentes associados aos diferentes erros de previsão (*i.e.:* prever que uma reserva será cancelada quando ela não é vs. prever um não-cancelamento quando ela é cancelada).

Vamos utilizar uma **Matriz de Confusão** para visualizar os **2 tipos de erro de classificação**. Primeiro, vamos criar a matriz manualmente para entendermos melhor o que está por trás da visualização:

In [ ]:
pd.crosstab(tb_pred_log["y_real"], tb_pred_log["y_previsto"], margins=True)


Agora vamos utilizar as funções `confusion_matrix` e `ConfusionMatrizDisplay` para visualizar nossa matriz:

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
cf = confusion_matrix(y_test, y_test_log)
cf_plot = ConfusionMatrixDisplay(cf)
cf_plot.plot(ax=ax)


### Precisão e Recall

A partir da Matriz de Confusão podemos entender as duas principais métricas do erro de classificação: **Precisão** e **Recall (ou *sensibilidade*)**. As duas métricas são utilizadas **em conjunto** para mitigar os riscos de sub-avaliarmos o erro na classe minoritária.

#### Precisão

A **Precisão** é uma medida de erro baseada **nas previsões positivas**, ou seja, **na segunda COLUNA da nossa matriz de confusão**. Ela é definida como:

$$ \text{Precision} = \frac{\text{Positivos Verdadeiros}}{\text{Positivos Verdadeiros} + \text{Falsos Positivos}}$$

Conceitualmente, a precisão mede o **quão corretas as previsões positivas do nosso modelo são**. Em nosso problema atual, ela mede **a % de previsões de cancelamentos que DE FATO serão cancelamentos**.

In [ ]:
from sklearn.metrics import precision_score, 

In [ ]:
np.round(precision_score(y_test, y_test_log), 2)


#### Recall (ou *sensibilidade*)

O **Recall** é uma medida de erro baseada nos **eventos positivos**, ou seja, na **segunda LINHA da nossa matriz de confusão**. Ela é definida como:

$$\text{Recall} = \frac{\text{Positivos Verdadeiros}}{\text{Positivos Verdadeiros} + \text{Falsos Negativos}} $$

Conceitualmente, o *recall* mede a **capacidade do nosso modelo de prever eventos positivos**. Em nosso exemplo, ele mede **a % de cancelamentos REAIS que nosso modelo prevê corretamente**.

In [ ]:
from sklearn.metrics import recall_score


In [ ]:
np.round(recall_score(y_test, y_test_log), 2)


### Combinando Precisão e Recall - F1 Score

As duas medidas apresentadas acima mede **qualidades opostas do nosso modelo**: a capacidade de fazer previsões positivas corretamente e a capacidade de prever todos os eventos positivos. Para combinar as duas medidas em uma só tradicionalmente utilizamos o F1-Score (a média harmonica entre as duas medidas):

$$ \text{F1-Score} = 2 \frac{\text{Precision} * \text{Recall}}{\text{Precision} + \text{Recall}}$$


**Quanto mais próximo o F1 está de 1, melhor nosso modelo!**

In [ ]:
from sklearn.metrics import f1_score


In [ ]:
f1_score(y_test, y_test_log)


# Classificador kNN

O primeiro algoritmo de ML que vamos analisar hoje é o `KNeighborsClassifier`, a implatanção na `sklearn` do algoritmo **kNN**. A idéia por trás do algoritmo kNN é simples: observações que são semelhantes entre si terão resultados semelhantes entre si.

Em nosso modelo, reservas com **lead time** e **adr** semelhantes terão taxas de cancelamento semelhantes. O algoritmo **kNN** busca as *k* observações mais próximas da observação que queremos prever e as utiliza para estimar a *probabilidade de cancelamento*. Um caso particular do kNN é quando k = 1: neste caso, buscamos a observação mais semelhante àquela que queremos realizar uma previsão e utilizamos o seu resultado como valor previsto!

![kNN](images/knn.png)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
knn_fit = KNeighborsClassifier(n_neighbors=50)
knn_fit.fit(X_train, y_train)


Agora, vamos utilizar nosso kNN com k = 50 para visualizar as probabilidades previstas para o conjunto de treinamento:

In [ ]:
tb_knn_fit = X_train.copy()
tb_knn_fit["pred_prob"] = knn_fit.predict_proba(X_train)[:, 1]
tb_knn_fit["is_cancelled"] = y_train
sns.scatterplot(data=tb_knn_fit, x="lead_time", y="pred_prob", hue="adr")


Assim como a probabilidade prevista, a superficie de decisão do algoritmo é extremamente irregular. Isso aponta para as qualidades e limitações do **kNN**:

1. Alta não-linearidade;
1. Dificuldade de generalização.

In [ ]:
fig = plt.figure(figsize=(10, 10))
plot_decision_regions(
    np.array(tb_knn_fit[["lead_time", "adr"]]),
    np.array(tb_knn_fit["is_cancelled"]),
    knn_fit,
    scatter_kwargs={"alpha": 0.001},
)


## Comparando o erro

Vamos comparar esse algoritmo rudimentar à nossa regressão logística utilizando as métricas que aprendemos mais cedo.

In [ ]:
y_test_knn = knn_fit.predict(X_test)
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].set_title("C.M. Reg.")
cf_log = confusion_matrix(y_test, y_test_log)
cf_log_plot = ConfusionMatrixDisplay(cf_log)
cf_log_plot.plot(ax=ax[0])
ax[1].set_title("C.M. kNN")
cf_knn = confusion_matrix(y_test, y_test_knn)
cf_knn_plot = ConfusionMatrixDisplay(cf_knn)
cf_knn_plot.plot(ax=ax[1])


In [ ]:
print("Erro Regressão:")
print(f"Precisão: {np.round(precision_score(y_test, y_test_log), 2)}")
print(f"Recall: {np.round(recall_score(y_test, y_test_log), 2)}")
print(f"F1-Score: {np.around(f1_score(y_test, y_test_log), 2)}\n")
print("Erro kNN:")
print(f"Precisão: {np.round(precision_score(y_test, y_test_knn), 2)}")
print(f"Recall: {np.round(recall_score(y_test, y_test_knn), 2)}")
print(f"F1-Score: {np.around(f1_score(y_test, y_test_knn), 2)}")


## Otimizando Hiperparâmetros

O número de vizinhos que consideramos no algoritmo **kNN** é o nosso primeiro contato com *hiperparâmetros*. **Hiperparâmetros** são como *definimos a forma que um algoritmo de ML* se comporta: ao contrário das regressões (linear e logística), um mesmo algoritmo pode ter diferentes utilizações - definidas pelo seu **conjunto de hiperparâmetros**. 

**SEMPRE** que utilizamos um algoritmo de ML **precisamos** otimizar os hiperparâmetros, ou seja, buscar o conjunto de hiperparâmetros com melhor performance para nosso problema. Vamos otimizar nosso algoritmo **kNN** para o problema atual começando pelo principal hiperparâmetro deste algoritmo: **k**.

In [ ]:
knn_fit = KNeighborsClassifier(n_neighbors=100)
knn_fit.fit(X_train, y_train)

y_test_knn = knn_fit.predict(X_test)
print(f"F1-Score: {np.around(f1_score(y_test, y_test_knn), 2)}")


A melhor forma de otimizar um hiperparâmetro é fazendo buscas exaustivas:

In [ ]:
for k in range(5, 56, 10):
    print(f"Testanto k = {k}")
    knn_fit = KNeighborsClassifier(n_neighbors=k)
    knn_fit.fit(X_train, y_train)

    y_test_knn = knn_fit.predict(X_test)
    print(f"F1-Score: {np.around(f1_score(y_test, y_test_knn), 2)}")


O segundo **hiperparâmetro** do algoritmo **kNN** é a *ponderação* (`weights = "distance"` na implantação da `sklearn`): se devemos considerar a distância dos k-vizinhos como forma de ponderar seus diferentes resultados.

In [ ]:
for k in range(5, 56, 10):
    print(f"Testanto k = {k}")
    knn_fit = KNeighborsClassifier(n_neighbors=k, weights="distance")
    knn_fit.fit(X_train, y_train)

    y_test_knn = knn_fit.predict(X_test)
    print(f"F1-Score: {np.around(f1_score(y_test, y_test_knn), 2)}")


Agora que fizemos uma busca *esparsa*, podemos construir uma busca **densa** mais específica:

In [ ]:
for k in range(2, 15):
    print(f"Testanto k = {k}")
    knn_fit = KNeighborsClassifier(n_neighbors=k, weights="distance")
    knn_fit.fit(X_train, y_train)

    y_test_knn = knn_fit.predict(X_test)
    print(f"F1-Score: {np.around(f1_score(y_test, y_test_knn), 2)}")


Para nosso problema parece que temos alguns pontos ótimos - vamos escolher um e construir a versão final de nosso algoritmo:

In [ ]:
knn_fit = KNeighborsClassifier(n_neighbors=10, weights="distance")
knn_fit.fit(X_train, y_train)

y_test_knn = knn_fit.predict(X_test)
print(f"F1-Score: {np.around(f1_score(y_test, y_test_knn), 2)}")


# Árvores de Decisão

O segundo algoritmo que veremos hoje é a **árvore de decisão**, implantada na `sklearn` através do `DecisionTreeClassifier`. O algoritmo da árvore de decisão funciona buscando **cortes** em nossos *features* que maximizem a diferença da nossa variável resposta nos grupos de observações em cada lado do corte.

Em cada etapa do algoritmo, buscamos cortes para todos os grupos criados até então. Este processo pode ser repetido até cada grupo conter apenas uma observação!

In [ ]:
from sklearn.tree import DecisionTreeClassifier


In [ ]:
dt_fit = DecisionTreeClassifier(max_depth=4)
dt_fit.fit(X_train, y_train)


In [ ]:
tb_dt_fit = X_train.copy()
tb_dt_fit["pred_prob"] = dt_fit.predict_proba(X_train)[:, 1]
tb_dt_fit["is_cancelled"] = y_train
sns.scatterplot(data=tb_dt_fit, x="lead_time", y="pred_prob", hue="adr")


In [ ]:
fig = plt.figure(figsize=(10, 10))
plot_decision_regions(
    np.array(tb_dt_fit[["lead_time", "adr"]]),
    np.array(tb_dt_fit["is_cancelled"]),
    dt_fit,
    scatter_kwargs={"alpha": 0.001},
)


## Visualizando a Árvore

Podemos construir uma visualização dos cortes construídos pelo agoritmo utilizando a função `plot_tree`:

In [ ]:
from sklearn.tree import plot_tree


In [ ]:
fig = plt.figure(figsize=(30, 10))
plot_tree(dt_fit, feature_names=X_train.columns, filled=True, fontsize=10);


Na árvore acima, podemos ver os diferentes **cortes** (grupos intermediários, com o *feature* e o valor do corte) e as **folhas** de nossa árvore (os últimos *nós* do grafo, contendo a proporção entre não-cancelamentos e cancelamentos).

## Comparando o Erro

Vamos utilizar a matriz de confusão e as métricas de classificação para comparar a performance de nossos modelos até agora:

In [ ]:
y_test_dt = dt_fit.predict(X_test)
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].set_title("C.M. Reg.")
cf_log = confusion_matrix(y_test, y_test_log)
cf_log_plot = ConfusionMatrixDisplay(cf_log)
cf_log_plot.plot(ax=ax[0])
ax[1].set_title("C.M. kNN")
cf_knn = confusion_matrix(y_test, y_test_knn)
cf_knn_plot = ConfusionMatrixDisplay(cf_knn)
cf_knn_plot.plot(ax=ax[1])
ax[2].set_title("C.M. Árvore de Decisão")
cf_dt = confusion_matrix(y_test, y_test_dt)
cf_knn_plot = ConfusionMatrixDisplay(cf_dt)
cf_knn_plot.plot(ax=ax[2])


In [ ]:
print("Erro Regressão:")
print(f"Precisão: {np.round(precision_score(y_test, y_test_log), 2)}")
print(f"Recall: {np.round(recall_score(y_test, y_test_log), 2)}")
print(f"F1-Score: {np.around(f1_score(y_test, y_test_log), 2)}\n")
print("Erro kNN:")
print(f"Precisão: {np.round(precision_score(y_test, y_test_knn), 2)}")
print(f"Recall: {np.round(recall_score(y_test, y_test_knn), 2)}")
print(f"F1-Score: {np.around(f1_score(y_test, y_test_knn), 2)}\n")
print("Erro Árvore de Decisão:")
print(f"Precisão: {np.round(precision_score(y_test, y_test_dt), 2)}")
print(f"Recall: {np.round(recall_score(y_test, y_test_dt), 2)}")
print(f"F1-Score: {np.around(f1_score(y_test, y_test_dt), 2)}")


## Otimizando Hiperparâmetros

Como todo algoritmo de ML, as árvores de decisão tem **hiperparâmetros** que precisam ser otimizados. Vamos começar com o principal hiperparâmetro das A.Ds: a **profundidade máxima** (`max_depth =` na implantação da `sklearn`).

A profundidade máxima determina o número máximo de cortes sucessivos que podemos fazer:

In [ ]:
f1_score_dt = []

for d in range(4, 150, 10):
    dt_fit = DecisionTreeClassifier(max_depth=d)
    dt_fit.fit(X_train, y_train)
    y_test_dt = dt_fit.predict(X_test)
    f1_score_dt.append(f1_score(y_test, y_test_dt))


In [ ]:
tb_depth_f1 = pd.DataFrame({"f1": f1_score_dt})
tb_depth_f1["depth"] = list(range(4, 150, 10))


Vamos visualizar a evolção do erro conforme aumentamos a profundidade máxima da A.D.:

In [ ]:
sns.scatterplot(data=tb_depth_f1, x="depth", y="f1")


Podemos ver que as melhores profundidades são acima de ~40 (com pouca melhoria entre 40 e 140). Vamos analisar outro hiperparâmetro crítico das árvores de decisão.

### Além do `max_depth`

O segundo parâmetro crítico em uma árvore de decisão é o tamanho mínimo de cada galho (em # de observações): ao aumentarmos o número mínimo de observações em cada corte reduzimos o número de cortes possíveis em partes menos densas de nossa árvore (ao contrário da profundidade, que controla a árvore de forma uniforme).

In [ ]:
f1_score_dt = []
hp_dt = []
depth_range = range(4, 150, 10)
mss_range = range(2, 500, 50)

for depth in depth_range:
    for mss in mss_range:
        dt_fit = DecisionTreeClassifier(max_depth=depth, min_samples_split=mss)
        dt_fit.fit(X_train, y_train)
        y_test_dt = dt_fit.predict(X_test)
        f1_score_dt.append(f1_score(y_test, y_test_dt))
        hp_dt.append((depth, mss))


In [ ]:
tb_hp_dt = pd.DataFrame(
    {
        "f1": f1_score_dt,
    }
)
tb_hp_dt[["depth", "min_sample_split"]] = hp_dt
tb_hp_dt.sort_values("f1", ascending=False)


In [ ]:
dt_fit = DecisionTreeClassifier(max_depth=124, min_samples_split=2)
dt_fit.fit(X_train, y_train)
y_test_dt = dt_fit.predict(X_test)


In [ ]:
y_test_dt = dt_fit.predict(X_test)
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].set_title("C.M. Reg.")
cf_log = confusion_matrix(y_test, y_test_log)
cf_log_plot = ConfusionMatrixDisplay(cf_log)
cf_log_plot.plot(ax=ax[0])
ax[1].set_title("C.M. kNN")
cf_knn = confusion_matrix(y_test, y_test_knn)
cf_knn_plot = ConfusionMatrixDisplay(cf_knn)
cf_knn_plot.plot(ax=ax[1])
ax[2].set_title("C.M. Árvore de Decisão")
cf_dt = confusion_matrix(y_test, y_test_dt)
cf_knn_plot = ConfusionMatrixDisplay(cf_dt)
cf_knn_plot.plot(ax=ax[2])


In [ ]:
print("Erro Regressão:")
print(f"Precisão: {np.round(precision_score(y_test, y_test_log), 2)}")
print(f"Recall: {np.round(recall_score(y_test, y_test_log), 2)}")
print(f"F1-Score: {np.around(f1_score(y_test, y_test_log), 2)}\n")
print("Erro kNN:")
print(f"Precisão: {np.round(precision_score(y_test, y_test_knn), 2)}")
print(f"Recall: {np.round(recall_score(y_test, y_test_knn), 2)}")
print(f"F1-Score: {np.around(f1_score(y_test, y_test_knn), 2)}\n")
print("Erro Árvore de Decisão:")
print(f"Precisão: {np.round(precision_score(y_test, y_test_dt), 2)}")
print(f"Recall: {np.round(recall_score(y_test, y_test_dt), 2)}")
print(f"F1-Score: {np.around(f1_score(y_test, y_test_dt), 2)}")


In [ ]:
fig = plt.figure(figsize=(10, 10))
plot_decision_regions(
    np.array(tb_dt_fit[["lead_time", "adr"]]),
    np.array(tb_dt_fit["is_cancelled"]),
    dt_fit,
    scatter_kwargs={"alpha": 0.001},
)


Na próxima aula veremos quais os problemas associados à superficies de decisão complexas: *overfitting*.